# python 简洁之美

## 一行代码交换a,b

In [1]:
a, b = 1, 4

a, b = b, a
a,b

(4, 1)

## 一行代码反转列表

In [2]:
[1,2,3][::-1] 

[3, 2, 1]

## 一行代码合并两个字典

In [3]:
{**{'a':1,'b':2}, **{'c':3}}

{'a': 1, 'b': 2, 'c': 3}

## 一行代码列表去重

In [4]:
set([1,2,2,3,3,3])

{1, 2, 3}

## 一行代码求多个列表中的最大值

In [5]:
max(max([ [1,2,3], [5,1], [4] ], key=lambda v: max(v))) 

5

## 一行代码生成逆序序列

In [6]:
list(range(10,-1,-1))

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

# python概览
《python编程》

## 任务

假设由于每种原因你现在需要去记录人们的信息。你想写个程序来这些人的详细信息。也就是说你想把这些记录存在数据库中，以便在电脑中永久存储这些信息。  

你可以找到现成的数据库管理软件来管理对应的数据库。但是你可以亲自编写这样一个程序，可以完全控制它的运行方式，于是就这样入python的坑.........

## 第一步 表示记录

如果我们想在数据库中存储记录，就要确定以什么样的数据类型来存储。通常使用列表和字典这样的内建对象类型就够用了。

### 试用 List

#### 新建记录

In [7]:
bob = ["Bob Smith", 42, 20000, "software"]
sue = ["Sue James", 30, 30000, "hardware"]

#### 通过对列表的访问，处理可以容易地处理记录

In [8]:
#　Bob　姓啥？
bob[0].split()[-1]

'Smith'

In [9]:
# 给 sue 涨薪 25%
sue[2] *= 1.25
sue

['Sue James', 30, 37500.0, 'hardware']

#### 数据库列表

为了把 Bob 和 Sue 存储在一起，我们可以简单地把他们放入一个新的列表里，形成一个简单的数据库

In [10]:
people = [bob, sue]
# 查看一个这个数据库的内容
for person in people:
    print(person)

['Bob Smith', 42, 20000, 'software']
['Sue James', 30, 37500.0, 'hardware']


通过操作数据库来处理记录

In [11]:
# 给每人涨薪 20%
for person in people:
    print(person[0].split()[-1])  # 打印姓氏
    person[2] *= 1.20   # 涨薪水

Smith
James


In [12]:
# 检查一下有没有涨薪水
for person in people:
    print(person[2])

24000.0
45000.0


#### 使用python迭代工具从记录中获取各种值。例如获取列表解析、map和生成器表达式

In [13]:
# 获取员工的薪水列表
# 列表解析
pays = [person[2] for person in people]
pays

[24000.0, 45000.0]

In [14]:
# map
pays = map((lambda x: x[2]), people)
list(pays)

[24000.0, 45000.0]

In [15]:
# # 查看公司应付员工薪水总额
# 生成器表达式， sum为内建函数
sum(person[2] for person in people)

69000.0

#### Field标签

通过对列表的位置来访问字段导致我们一直需要记住列表每个位置的含义，这样编码麻烦而且代码可读性也差，最好把字段名和字段的值关联起来，也就是咱们给列表的每个位置起个名字。

In [16]:
NAME, AGE, PAY, JOB = range(4)
kobe = ["Kobe Bryant", 60, 60000, "spokesperson"]

In [17]:
# 查看姓名
kobe[NAME]

'Kobe Bryant'

In [18]:
# 查看职位
JOB, kobe[JOB]  

(3, 'spokesperson')

这样兄弟萌就解决了可读性问题，但是我们发现数据记录和字段名并非是关联在一起的，这样随着时间的前进，后来的编码人员对数据结构进行变化时，如果忘了对字段变量做相应的维护的话，它们将会失去同步。  

而且字段名是独立的变量，没有从记录到其字段名的直接映射。  

我们可以尝试利用元组的列表来解决这个问题，其中每个元组保存字段名和值，更加理想的是，使用列表的列表允许进行更新的，（元组是不可变的）

In [19]:
bob = [['name', "Bob Smith"], ["age", 42], ["pay", 30000]]
sue = [['name', "Sue James"], ["age", 46], ["pay", 40000]]
people = [bob, sue]

但是这样我们依然需要通过列表的index值来获取数据。

In [20]:
# 收集人员姓名
[person[0][1] for person in people]

['Bob Smith', 'Sue James']

实际上仅仅是增加了一层额外的位置索引。更好的办法是，可以在循环中检查字段名字来找到我们想要的属性

In [21]:
for person in people:
    for (name, value) in person:
        if name == "name" :
            print(value)

Bob Smith
Sue James


我们可以抽取其中的逻辑写个函数接口来做项工作，以后调用函数进行复用就可以了

In [22]:
def field(record, label):
    """
    record: 记录变量名
    label： 字段名
    """
    for(fname, fvalue) in record:
        if fname == label:  # 根据名字来寻找字段名
            return fvalue

In [23]:
# 获取姓名
field(bob, "name")

'Bob Smith'

In [24]:
# 打印所有年龄
for rec in people:
    print(field(rec, "age"))   #简化了代码，提升了可读性

42
46


如果继续探索下去，最终会实现一套将字段名映射到字段值的完善的记录接口函数。

#### 持久存储问题

列表的确可以用作我们的人员数据库，而且的确它也满足一些小程序的需要。但是他是在内存中，一旦python的进程退出，这些数据就会从内存中消失。

### 试用字典

#### 新建字典数据表示

虽然列表的记录表示方式能够使用，但是我们知道内建的字典对象在关联属性的名字和值方面会更高效和方便

In [25]:
bob = {"name":"Bob Smith", "age":42, "pay":30000, "job":"dev"}
sue = {"name":"sue James", "age":46, "pay":40000, "job":"mng"}

In [26]:
# 相比list我们不再需要记住index值偏移量对应的含义
bob["name"]

'Bob Smith'

In [27]:
# 涨薪  
print(sue["pay"])
sue["pay"] *= 1.10
print(sue["pay"])

# 代码可读性也大大地好
# 字段名和值之间的映射关系也稳定，也不用担心它们之间会失联

40000
44000.0


#### 其他建立字典的方法

可以使用关键词参数和类型构造函数，主要所有的键是字符串

In [28]:
bob = dict(name="Bob Smith", age=42, pay=30000, job="dev")
sue = dict(name="Sue Jones", age=45, pay=40000, job="hdw")
bob

{'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}

或者一次一个字段地填写（字典的键是伪随机排列的呦）

In [29]:
wade = {}

In [30]:
wade["name"] = "D.Wade"
wade

{'name': 'D.Wade'}

In [31]:
wade["age"] = "40"
wade

{'name': 'D.Wade', 'age': '40'}

或者用zip函数将 名/值 两个列表链在一起

In [32]:
keys = ["name", "age", "pay", "job"]
values = ["L.J", "35", "35000000$", "basketball player"] 

# 1
list(zip(keys, values))

[('name', 'L.J'),
 ('age', '35'),
 ('pay', '35000000$'),
 ('job', 'basketball player')]

In [33]:
# 2
dict(zip(keys, values))

{'name': 'L.J', 'age': '35', 'pay': '35000000$', 'job': 'basketball player'}

甚至可以通过一个键序列和所有键的可选初始值来创造字典（便于初始化空字典）

In [34]:
# 先把结构创建出来，然后可以在慢慢填入
fields = {"name", "age", "job"}
record = dict.fromkeys(fields, "?")
record

{'age': '?', 'name': '?', 'job': '?'}

#### 字典列表数据库

不管怎样实现字典，仍然需要把基于字典的数据记录会聚在一起，形成数据库。我们依然使用列表

In [35]:
bob

{'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}

In [36]:
sue

{'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'hdw'}

In [37]:
people = [bob, sue]
for person in people:
    print(person["name"], person["pay"])

Bob Smith 30000
Sue Jones 40000


对数据库可以使用迭代工具

In [38]:
# 人员有哪些
names = [person["name"] for person in people]
names

['Bob Smith', 'Sue Jones']

In [39]:
# map
list(map((lambda x: x["name"]), people))

['Bob Smith', 'Sue Jones']

In [40]:
# 总共应付人员薪水
sum(person["pay"] for person in people)

70000

In [41]:
db = {}                 # 数据库
db["bob"] = bob         # 引用字典的字典
db["sue"] = sue

由于字典是python对象，所以数据库中的记录可以用python语法进行增删改查

In [42]:
for person in people:
    person["pay"] *= 1.10  # 为所有人员加薪

#### 嵌套结构

In [43]:
# 现在抽取姓氏时
for person in people:
    print(person["name"].split()[-1])

Smith
Jones


通过再进一步结构化我们的记录，可以避免上述列子中这样的抽取姓氏的代码

In [44]:
# 将字典、列表、元组嵌套在另一个字典中
bob2 = {"name":{"first":"bob", "last":"Smith"},
        "age":42,
        "job":["software", "dancer"],
        "pay":(30000, 20000),
       }

In [45]:
# 获取全名
bob2["name"]

{'first': 'bob', 'last': 'Smith'}

In [46]:
# 获取bob的姓
bob2["name"]["last"]

'Smith'

In [47]:
# bob 的所有工作
for job in bob2["job"] : print(job) 

software
dancer


In [48]:
# bob 又增加了一个工作
bob2["job"].append("lol player")

In [49]:
bob2["job"]

['software', 'dancer', 'lol player']

#### 字典的字典

我们对基于字典的人员数据库做最后一次改进，可以从字典中多挖掘一些字典的好处。  

我们使用字典表示数据库本身，

In [50]:
bob = dict(name="Bob Smith", age=42, pay=30000, job="dev")
sue = dict(name="Sue Jones", age=45, pay=40000, job="hdw")

In [51]:
db = {} # 建立一个数据库

In [52]:
# 引用字典的字典
db["bob"] = bob 
db["sue"] = sue

In [53]:
db["bob"]["name"]

'Bob Smith'

In [54]:
db["sue"]["pay"] * 1.1  # 涨薪

44000.0

In [55]:
# 这样的结构可以直接访问一条记录，而不必循环中寻找到它。
db #查看数据库

{'bob': {'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'},
 'sue': {'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'hdw'}}

In [56]:
# python的pprint可以让输出更易于阅读
import pprint
pprint.pprint(db)

{'bob': {'age': 42, 'job': 'dev', 'name': 'Bob Smith', 'pay': 30000},
 'sue': {'age': 45, 'job': 'hdw', 'name': 'Sue Jones', 'pay': 40000}}


如果仍然需要逐条记录地访问数据库，可以使用for循环和字典的迭代器

In [57]:
# 迭代起
x = [db[key]["name"] for key in db]
y = [rec["name"] for rec in db.values()]
print(x, y)

['Bob Smith', 'Sue Jones'] ['Bob Smith', 'Sue Jones']


#### 数据库增删改查

In [58]:
#　数据库的记录总数
len(db)

2

In [59]:
# 增 - 只要把新的记录赋给一个新的键即可
db["tom"] = dict(name="Tom", age=50, job=None, pay=0)
pprint.pprint(db)

{'bob': {'age': 42, 'job': 'dev', 'name': 'Bob Smith', 'pay': 30000},
 'sue': {'age': 45, 'job': 'hdw', 'name': 'Sue Jones', 'pay': 40000},
 'tom': {'age': 50, 'job': None, 'name': 'Tom', 'pay': 0}}


In [60]:
# 查看数据库有哪些人员记录
list(db.keys())

['bob', 'sue', 'tom']

In [61]:
# 类似 SQL 的条件查询
[rec["name"] for rec in db.values() if rec["age"] >= 45] 

['Sue Jones', 'Tom']

#### 持久存储问题

虽然我们的数据库仍然是内存的临时对象，当python进程死亡时，这些数据也随之消失，但是这种字典的结构与永久性对象的系统结构非常切合--shelve(英文语法上应该是shelf，但是python模块命名和术语用的shelve)

## 第二步  持久化存储记录

由于前面的所有对象是在内存中的都是暂时的，所以为了人员信息持久化，我们需要将对象保存在某种类型的文件中。

### 使用格式化文件

程序运行时保存数据的一种方法是把所有数据以格式化的方法写入一个简单的文本文件中。只要保存和载入的工具在所选格式上达成一致就可以

#### 生成测试数据的接口（可设计为脚本模块，命令行运行）

In [62]:
# 可以单独制作成一个py文件作为脚本，通过__main__入口调用
def initdata():
    #　记录       
    bob = dict(name="Bob Smith", age=42, pay=30000, job="dev")
    sue = dict(name="Sue Jones", age=45, pay=40000, job="hdw")
    tom = dict(name="Tom Sun", age=48, pay=0, job="")

    # db
    db = {}
    db["bob"] = bob
    db["sue"] = sue
    db["tom"] = tom
    
    return db

# 生成
initdata() 

{'bob': {'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'},
 'sue': {'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'hdw'},
 'tom': {'name': 'Tom Sun', 'age': 48, 'pay': 0, 'job': ''}}

#### 数据格式化脚本

建立一些 标识变量

In [63]:
dbfilename = "people-file"   # 文件名
ENDDB = "enddb."     # 数据库分割标识        
ENDREC = "endrec."   # 记录间分割标识
RECSEP = "=>"        # 记录的 键/值 映射标识

数据格式化保存接口

In [64]:
def storeDb(db, dbfilename=dbfilename):
    "将数据格式化保存为不同文件"
    dbfile = open(dbfilename, "w")
    for key in db:
        print(key, file=dbfile)  # dbfile.write(key + "\n") 同效
        for (name, value)in db[key].items():
            print(name + RECSEP + repr(value), file=dbfile)  #　repr将其当作代码执行
        print(ENDREC, file=dbfile)
    print(ENDDB, file=dbfile)
    dbfile.close()

In [65]:
# 将测试数据持久化存储
storeDb(initdata())  

In [66]:
# 浏览一下这个文件
for line in open("people-file"):
    print(line, end="")

bob
name=>'Bob Smith'
age=>42
pay=>30000
job=>'dev'
endrec.
sue
name=>'Sue Jones'
age=>45
pay=>40000
job=>'hdw'
endrec.
tom
name=>'Tom Sun'
age=>48
pay=>0
job=>''
endrec.
enddb.


从文件解析数据并重构数据库对象

In [67]:
def loadDb(dbfilename=dbfilename):
    "解析数据， 重构数据库"
    db = {}         # 数据库的空间
    rec = {}        # 记录的缓存空间
    recname = ""    # 数据库的  键
    with open(dbfilename, "r") as dbfile:
        for line in dbfile :
            line = line.strip("\n")        
            if line != ENDDB:  
                if line != ENDREC:
                    if RECSEP  in line : 
                        name, value = line.split(RECSEP)
                        rec[name] = eval(value)    # 往记录添加数据
                       
                        # print(recname)
                    else:
                        recname = line  #　记录名
                        db[recname] = rec
                    # print(rec)
                else:
                    rec = {}    # 一旦当前记录结束，缓存置空，
                    continue    # 跳过 endrec. 这一行继续执行
            else:
                return db

In [68]:
db = loadDb()  # 解析文件并构建数据库

对重构的数据库可以进行增删改查，并重新写入文件

In [69]:
db["tom"]["job"] = "mng"
db["tom"]["pay"] = 50000
db["tom"]

{'name': 'Tom Sun', 'age': 48, 'pay': 50000, 'job': 'mng'}

In [70]:
storeDb(db)  # 将更改厚的数据库写入文件
loadDb()     # 检查一下是否更改生效


{'bob': {'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'},
 'sue': {'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'hdw'},
 'tom': {'name': 'Tom Sun', 'age': 48, 'pay': 50000, 'job': 'mng'}}

到此为止，就算计算机关机都不会丢失数据了，因为我们已经将数据库对象格式化保存到硬盘里的文件里了，我们可以靠专用的脚本载出更改和存入这些对象了。这样我萌就算把对象持久化了。

###  使用 pickle文件

虽然我们格式化文本文件结构是可行的，但是他有些较大的局限性。  

首先为了获取一条记录或者更改一条记录就需要读取和写入整个数据库，这样会让程序越来复杂越来越慢。  

其次，我们格式化的方法为例，用 => 做分割符的办法，当文本内容里出现 => 这样的内容，就会发生冲突，导致这种格式化的结构失败。虽然我们可以采用XML的方式来结构化，但是创建XML和解析XML的工作是很烦银的 

如果有一种通用的工具，它只需一步就可以将任意类型的Python对象转化成可以保存在文件中的格式，而你不需要关心，文件的格式化结构，不必为了解析文件而费心，那就好了。这正是Pickle设计的初衷。

Pickle模块将内存中的Python对象转化成序列化的字节流，这是一种可以写入任何类似文件对象的字节串。  
Pickle模块也可以根据序列化的字节流重新构建原来内存中的对象。将文件转换成原来的那个对象。  

可以说 Pickle模块取代了专有的数据格式。

In [71]:
# 存
import pickle 

db = initdata() # 生成测试数据
dbfile = open("people-pickle", "wb")   # 用字节的方式进行写
pickle.dump(db, dbfile)  # 把 db 对象pickle进文件里
dbfile.close()

In [72]:
# 取
import pprint

with open("people_pickle", "rb") as dbfile:   # 用字节的方式进行读 
     db_ = pickle.load(dbfile)  # 读取           
     pprint.pprint(db_)

{'bob': {'age': 42, 'job': 'dev', 'name': 'Bob Smith', 'pay': 30000},
 'sue': {'age': 45, 'job': 'hdw', 'name': 'Sue Jones', 'pay': 40000},
 'tom': {'age': 48, 'job': 'mng', 'name': 'Tom Sun', 'pay': 50000}}


In [73]:
# 更新
db["sue"]["pay"] *= 1.10

# 再写入
dbfile = open("people-pickle", "wb")  # 用字节的方式进行写
pickle.dump(db, dbfile)  # 把 db 对象pickle进文件里
dbfile.close()

我们观察到当记录在内存中被修改后是将整个数据库重新写入文件，这和我们自己手动格式化文件的方式一样的，当数据库比较庞大时，这些操作会变得很慢。暂时保留这个问题

Pickle和它的数据格式也可以处理shelve和ZODB数据库，pickle的类实例为被存储的对象提供了数据和操作。  

实际上Pickle更加通用，只要对象具有文件兼容的接口，就可以通过pickle将内存中的Python对象转换成多种媒介存储。  

可以使用网络套接字（Socket）可以将Pickle过的Python对象通过网络进行传输，甚至作为SOAP和XML-RPC这样的大型协议的代替。

### 每条记录使用一个pickle文件

前面提到的潜在的缺点，当数据库非常大是变得很慢，慢的原因是更新操作一条记录就要将整个数据库重新读取存入。  

我们可以通过存储数据库中的一条记录到一个普通文件的方式来改进。以记录本身的键为文件名

1. 将每条记录存到对应的文件中

In [74]:
import pickle

for (key, record) in [("bob", bob), ("sue", sue)]:
    with open(key+".pkl", "wb") as recfile:
        pickle.dump(record, recfile)

2  使用glob模块获取整个数据库。它根据文件拓展名来获取这个目录下所有该拓展名的文件。如果需要载入单个记录，可以打开对应的文件并使用pickle反序列化。为了得到一条记录，我们只需要载入一个记录而不需要整个数据库

In [75]:
import glob

# 查看整个数据库
for filename in glob.glob("*.pkl"):    
    with open(filename, "rb") as recfile:
        record = pickle.load(recfile)
        print(filename, "=>\n",record)

#　查看单个记录
suefile = open("sue.pkl", "rb")
print("sue.pkl['name']", "=>\n", pickle.load(suefile)["name"])
suefile.close()

bob.pkl =>
 {'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}
sue.pkl =>
 {'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'hdw'}
sue.pkl['name'] =>
 Sue Jones


3  当更新数据库，从对应文件抓取一条记录，在内存中进行修改，然后重新写入它的pickle文件，不用为了更新某段记录而对整个数据库进行操作

In [76]:
suefile = open("sue.pkl", "rb")
sue = pickle.load(suefile)
suefile.close()

sue["pay"] *= 1.10  # 给sue涨薪10%
suefile = open("sue.pkl", "wb")
pickle.dump(sue, suefile)
suefile.close()

# 查看该记录是否更新成功
suefile = open("sue.pkl", "rb")
print("sue.pkl['pay']", "=>\n", pickle.load(suefile)["pay"])
suefile.close()

sue.pkl['pay'] =>
 44000.0


上述方法数据库的中的key变为了实际的文件名。也就是说文件系统成为了高层字典，通过文件名可以直接访问每条记录

### 使用shelves 
假设你的文件系统可以处理任意多需要的文件，将一条记录pickle成一个大文件。避免了每次更新和操作时都需要载入和存储整个数据库，如果我们用键来访问记录，Python表批准库提供了一中更高层次的工具：shelves

它自动地将对象pickle进和pickle出健访问文件系统。它们很像必须打开着的字典，在程序退出时进行持久化。shelves使用起来像是必须打开着的一个字典。 

1. 创建数据库

In [77]:
import shelve

db = shelve.open("people-shelve") 
db["bob"] = bob
db["sue"] = sue
db.close()

2. 访问数据库, 依旧是字典的字典

In [78]:
# 像字典一样通过键来访问
db = shelve.open("people-shelve")
for key in db:
    print(key, "=>\n", db[key]) 

bob =>
 {'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}
sue =>
 {'name': 'Sue Jones', 'age': 45, 'pay': 44000.0, 'job': 'hdw'}
tom =>
 {'name': 'Tom', 'age': 50, 'job': None, 'pay': 0}


3. 更新

In [79]:
# 需要加入的新数据
tom= dict(name="Tom", age=50, job=None, pay=0)

# 打开数据库
db = shelve.open("people-shelve") # 缺省的方式
# 更改记录
sue = db["sue"]   
sue["pay"] *= 1.1

# 手动写回的原因是缺省需求，可以在open中定义关闭shelve时写回，但是这么干会消耗内存使得关闭变慢
# 缺省：需要时创建，否则以可读可写方式打开 
db["sue"] = sue

# 添加新记录
db["tom"] = tom
db.close() # 关闭shelve、关闭缓存通道

# 可以将数据库操作单独制作成脚本文件，通过运行相应程序 自动对本地数据库操作并将变化状态持久化地保存在文件中，

## 第三步 走进OOP

面向对象的方式能使得实现类似操作时避免代码的重复编写，提高代码的复用，代码的调试和维护的难度也降低了。

### 构建类

实例化类的方式来实现记录数据库

In [80]:
class Person:
    def __init__(self, name, age, pay=0, job=None):
        self.name = name
        self.age = age
        self.pay = pay
        self.job = job

构建记录（类的实例）

In [81]:
bob = Person("Bob Smith", 42, 30000, "software")
sue = Person("Sue Jones", 45, 35000, "hartware")
bob.name.split()[-1] # 获取姓氏

'Smith'

更新记录

In [82]:
# 涨薪
sue.pay *= 1.1
sue.pay

38500.0

这个类还不是一个数据库，将类的实例放入列表和字典，构成一个整体

In [83]:
bob = Person("Bob Smith", 42, 30000, "software")
sue = Person("Sue Jones", 45, 35000, "hartware")

people = [bob, sue]  # 构成一个数据库列表
for person in people:
    print(person.name, person.pay)

Bob Smith 30000
Sue Jones 35000


查询

In [84]:
x = [(person.name, person.age, person.pay,  person.job) for person in people]
x

[('Bob Smith', 42, 30000, 'software'), ('Sue Jones', 45, 35000, 'hartware')]

SQL 风格查询

In [85]:
[rec.name for rec in people if rec.age >= 45]

['Sue Jones']

### 为类添加行为

现在我们们实例化类仅仅是得到数据。它使用对象的属性替代字典的键，但是并没有原来的那些操作方法，为了利用类的功能完成这些操作，我们为类添加一些行为。

In [86]:
class Person:
    def __init__(self, name, age, pay=0, job=None):
        self.name = name
        self.age = age
        self.pay = pay
        self.job = job
    
    # 捕获姓氏行为
    def get_LastName(self):
        return self.name.split()[-1]
    
    # 涨薪行为
    def giveRaise(self, percent=0.1):
        self.pay *= (1.0 + percent)

In [87]:
# 测试一下
bob = Person("Bob Smith", 42, 30000, "software")
sue = Person("Sue Jones", 45, 35000, "hartware")

bob.get_LastName()

'Smith'

In [88]:
# 一直涨薪一直爽
print(sue.pay)
sue.giveRaise()
print(sue.pay)

35000
38500.0


### 继承  通过人员类型编写子类

构建管理层子类， 可以单独地对管理层进行一些定制化的行为，而不会影响到父类（所有人员）

In [89]:
# 比如管理层除了正常涨薪还有 额外百分比的奖金的话 哎
class Manager(Person):
    
    # 重写涨薪行为
    def giveRaise(self, percent=0.1, bonus=0.1) :
        self.pay *= (1.0 + percent +bonus)

In [90]:
tom = Manager(name="Tom Doe", age=50, pay=50000)
tom.get_LastName() # 父类的功能

'Doe'

汤姆因为是Manager类的实例，给他涨薪的时候，除了父类的10%还有自定义的奖金

In [91]:
print(tom.pay)
# 公司默认奖金10%，但是这次表现突出给奖金20%，加上基础所有人员涨薪10%，这样就是30%了哦
tom.giveRaise(bonus=.20) 
tom.pay


50000


65000.0

这样可以在父类中全局控制所有记录的共有属性，在子类中管理各个类型的特有行为，除了可维护性高，也不会在记录间相互干扰。

###  重构代码

####  增强方法

我们发现涨薪计算在两个地方重复出项了（在两个类中）。我们对子类的涨薪方法可以通过增强继承来实现，而不是完全替代父类方法（完全重写），从而达到去除代码中的冗余的效果。

In [92]:
class Manager(Person):
    
    def giveRaise(self, percent=0.1, bonus=0.1) :
        Person.giveRaise(self, percent + bonus)
# 1.调用父类的方法显式地传入self参数
# 2. 虽然重新定义了该方法，但我们调用的是通用的方法，可以降低冗余（业务逻辑只出现一次）
# 3.实践中易于启动父类的构造器，这样在父类中重新定义了percent参数设置时，子类中会启动父类构造器，自动生效。

####  格式化显示

为了获得更多面向对象的乐趣，咱们要为people类增加一些操作符重载。

In [93]:
# 这是还没重载时的输出，默认的只有地址信息，而且可读性也是挺差的
tom = Manager("Tom Jams", 60, 50000)
print(tom)

In [94]:
class Person:
    def __init__(self, name, age, pay=0, job=None):
        self.name = name
        self.age = age
        self.pay = pay
        self.job = job
    
    # 操作符重载
    def __str__(self):
        # 当打印 Person 实例时 python 会自动调用这个方法滴，默认的返回值信息少的很
        # 自定义返回值,对当对象需要作为整体打印时，比实例缺省的显示好得多
        return "<%s => %s: %s, %s>" % (self.__class__.__name__, self.name, self.job, self.pay)
    # 可以编写 __add__ 使得 + 表达式自动调用 giveraise方法，代价是代码可读性变差，不知到具体你 + 了啥？
    # 还有更多的操作符可以重载 __repr__、 __call__、 __iter__ 等等...
    
    
    def get_LastName(self):
        return self.name.split()[-1]
    
    def giveRaise(self, percent=0.1):
        self.pay *= (1.0 + percent)

class Manager(Person):
    
    def giveRaise(self, percent=0.1, bonus=0.1) :
        Person.giveRaise(self, percent + bonus)

In [95]:
# 测试一下打印时输出有啥改变， 呦！
tom = Manager("Tom Jams", 60, pay=6000, job="manager")
print(tom)

<Manager => Tom Jams: manager, 6000>


#### 自定义构造函数

我们又发现了一个问题：  tom = Manager("Tom Jams", 60, pay=6000, job="manager")  
这个类实例化的语句中命名参数： job="manager" 存在着冗余，想想看这是manger类的实例，那job这个命名参数有点废话的赶脚嘛  
比较合理的是为manager类提供一个显式的构造函数，自动填写job参数。

In [96]:
# 说干就干
class Manager(Person):
    
    def __init__(self, name, age, pay):
        Person.__init__(self, name, age, pay, job="Manager") # 这里调用父类的构造器，并自动填写job

    def giveRaise(self, percent=0.1, bonus=0.1) :
        Person.giveRaise(self, percent+bonus)

In [97]:
# 测试一下  格式化输出
bob = Person("Bob Smith", 44)    # 还没为bon分配工作
sue = Person("Sue Jones", 47, 40000, "Hardware") # 分配了硬件工程师（可以再写个硬件工程师子类）
tom = Manager(name="Tom Doe", age=50, pay=50000) # 分配到经理
for obj in (bob, sue, tom):
    obj.giveRaise(.10)
    print(obj)

<Person => Bob Smith: None, 0.0>
<Person => Sue Jones: Hardware, 44000.0>
<Manager => Tom Doe: Manager, 60000.0>


### 增加持久化

我们现在已经把记录和对其的操作封装进了类中。我们可以把她们存储在一个单记录的pickle文件中，而基于shelve的存储媒介也能很好的实现我们的想法。并且更容易编码。

In [98]:
# 创建记录（实例） 
bob = Person("Bob Smith", 44)
sue = Person("Sue Jones", 47, 40000, "Hardware")
tom = Manager(name="Tom Doe", age=50, pay=50000)

# 创建存储记录(实例)的shelve，也就是咱们的顶层数据库了
db = shelve.open("class-shelve")

# 然后将 记录 赋给新建的shelve文件的 键
db["bob"] = bob
db["sue"] = sue
db["tom"] = tom
db.close()

In [99]:
# 查看数据库
db = shelve.open("class-shelve")
for key in db:
    print(key, " =>\n ", db[key].name, db[key].pay)

bob  =>
  Bob Smith 0
sue  =>
  Sue Jones 40000
tom  =>
  Tom Doe 50000


In [100]:
#  对数据库中的实例进行操作
bob = db["bob"]
print(bob.get_LastName()) 


# 可以直接操作
# 不需要导入实例的相关类，pickle系统会自动定位相关类，只要类在模块搜索路径上。

Smith


由于类和他的实例在shelve中是被分别存储的，所以可以在外部通过修改类来改变实例被载入时的解释方法。  

可以对shleve数据库中存储的实例进行更改，因为这些对象都是持久化在shelve中的/。也可以通过python交互命令打开并查看shelve，尽管其存储具有长期性，但是shelve其实真正上只是一种包含了python对象的对象（它本身也是一个shelve实例）

### 其他数据库选项

shelve功能齐全，虽然它不是关系型数据库（我们存储的是对象，不是表），但是对于一些类型的程序这已经够用了

如果需要更多的功能，我们可以将这个应用移植到一些更强大的工具上。

# 系统编程

## 获取操作系统的一些信息 （摘自网络）

In [101]:
import platform 

'''
    python中，platform模块给我们提供了很多方法去获取操作系统的信息
    如：
        import platform
        platform.platform()        #获取操作系统名称及版本号，'Linux-3.13.0-46-generic-i686-with-Deepin-2014.2-trusty'  
        platform.version()         #获取操作系统版本号，'#76-Ubuntu SMP Thu Feb 26 18:52:49 UTC 2015'
        platform.architecture()    #获取操作系统的位数，('32bit', 'ELF')
        platform.machine()         #计算机类型，'i686'
        platform.node()            #计算机的网络名称，'XF654'
        platform.processor()       #计算机处理器信息，''i686'
        platform.uname()           #包含上面所有的信息汇总，('Linux', 'XF654', '3.13.0-46-generic', '#76-Ubuntu SMP Thu Feb 26 18:52:49 UTC 2015', 'i686', 'i686')

        还可以获得计算机中python的一些信息：
        import platform
        platform.python_build()
        platform.python_compiler()
        platform.python_branch()
        platform.python_implementation()
        platform.python_revision()
        platform.python_version()
        platform.python_version_tuple()
'''

# global var
#是否显示日志信息
SHOW_LOG = True

def get_platform():
    '''获取操作系统名称及版本号'''
    return platform.platform()

def get_version():
    '''获取操作系统版本号'''
    return platform.version()

def get_architecture():
    '''获取操作系统的位数'''
    return platform.architecture()

def get_machine():
    '''计算机类型'''
    return platform.machine()

def get_node():
    '''计算机的网络名称'''
    return platform.node()

def get_processor():
    '''计算机处理器信息'''
    return platform.processor()

def get_system():
    '''获取操作系统类型'''
    return platform.system()

def get_uname():
    '''汇总信息'''
    return platform.uname()

def get_python_build():
    ''' the Python build number and date as strings'''
    return platform.python_build()

def get_python_compiler():
    '''Returns a string identifying the compiler used for compiling Python'''
    return platform.python_compiler()

def get_python_branch():
    '''Returns a string identifying the Python implementation SCM branch'''
    return platform.python_branch()

def get_python_implementation():
    '''Returns a string identifying the Python implementation. Possible return values are: ‘CPython’, ‘IronPython’, ‘Jython’, ‘PyPy’.'''
    return platform.python_implementation()

def get_python_version():
    '''Returns the Python version as string 'major.minor.patchlevel'
    '''
    return platform.python_version()

def get_python_revision():
    '''Returns a string identifying the Python implementation SCM revision.'''
    return platform.python_revision()

def get_python_version_tuple():
    '''Returns the Python version as tuple (major, minor, patchlevel) of strings'''
    return platform.python_version_tuple()

def show_os_all_info():
    '''打印os的全部信息'''
    print('获取操作系统名称及版本号 : [{}]'.format(get_platform()))
    print('获取操作系统版本号 : [{}]'.format(get_version()))
    print('获取操作系统的位数 : [{}]'.format(get_architecture()))
    print('计算机类型 : [{}]'.format(get_machine()))
    print('计算机的网络名称 : [{}]'.format(get_node()))
    print('计算机处理器信息 : [{}]'.format(get_processor()))
    print('获取操作系统类型 : [{}]'.format(get_system()))
    print('汇总信息 : [{}]'.format(get_uname()))

def show_os_info():
    '''只打印os的信息，没有解释部分'''
    print(get_platform())
    print(get_version())
    print(get_architecture())
    print(get_machine())
    print(get_node())
    print(get_processor())
    print(get_system())
    print(get_uname())

def show_python_all_info():
    '''打印python的全部信息'''
    print('The Python build number and date as strings : [{}]'.format(get_python_build()))
    print('Returns a string identifying the compiler used for compiling Python : [{}]'.format(get_python_compiler()))
    print('Returns a string identifying the Python implementation SCM branch : [{}]'.format(get_python_branch()))
    print('Returns a string identifying the Python implementation : [{}]'.format(get_python_implementation()))
    print('The version of Python ： [{}]'.format(get_python_version()))
    print('Python implementation SCM revision : [{}]'.format(get_python_revision()))
    print('Python version as tuple : [{}]'.format(get_python_version_tuple()))

def show_python_info():
    '''只打印python的信息，没有解释部分'''
    print(get_python_build())
    print(get_python_compiler())
    print(get_python_branch())
    print(get_python_implementation())
    print(get_python_version())
    print(get_python_revision())
    print(get_python_version_tuple())
      
def test():
    print('操作系统信息:')
    if SHOW_LOG:
        show_os_all_info()
    else:
        show_os_info()
    print('#' * 50)
    print('计算机中的python信息：')
    if SHOW_LOG:
        show_python_all_info()
    else:
        show_python_info()

def init():
    global SHOW_LOG
    SHOW_LOG = True  # 需不要解释性
    
def main():
    init()
    test()

if __name__ == '__main__':
    main()

操作系统信息:
获取操作系统名称及版本号 : [Windows-7-6.1.7601-SP1]
获取操作系统版本号 : [6.1.7601]
获取操作系统的位数 : [('32bit', 'WindowsPE')]
计算机类型 : [x86]
计算机的网络名称 : [DE142]
计算机处理器信息 : [x86 Family 6 Model 60 Stepping 3, GenuineIntel]
获取操作系统类型 : [Windows]
汇总信息 : [uname_result(system='Windows', node='DE142', release='7', version='6.1.7601', machine='x86', processor='x86 Family 6 Model 60 Stepping 3, GenuineIntel')]
##################################################
计算机中的python信息：
The Python build number and date as strings : [('default', 'Apr 24 2019 13:20:13')]
Returns a string identifying the compiler used for compiling Python : [MSC v.1915 32 bit (Intel)]
Returns a string identifying the Python implementation SCM branch : []
Returns a string identifying the Python implementation : [CPython]
The version of Python ： [3.7.3]
Python implementation SCM revision : []
Python version as tuple : [('3', '7', '3')]


## os

* os.remove(‘path/filename’) 删除文件
* os.rename(oldname, newname) 重命名文件
* os.walk() 生成目录树下的所有文件名
* os.chdir('dirname') 改变目录
* os.mkdir/makedirs('dirname')创建目录/多层目录
* os.rmdir/removedirs('dirname') 删除目录/多层目录
* os.listdir('dirname') 列出指定目录的文件
* os.getcwd() 取得当前工作目录
* os.chmod() 改变目录权限
* os.path.basename(‘path/filename’) 去掉目录路径，返回文件名
* os.path.dirname(‘path/filename’) 去掉文件名，返回目录路径
* os.path.join(path1[,path2[,...]]) 将分离的各部分组合成一个路径名
* os.path.split('path') 返回( dirname(), basename())元组
* os.path.splitext() 返回 (filename, extension) 元组
* os.path.getatime\ctime\mtime 分别返回最近访问、创建、修改时间
* os.path.getsize() 返回文件大小
* os.path.exists() 是否存在
* os.path.isabs() 是否为绝对路径
* os.path.isdir() 是否为目录
* os.path.isfile() 是否为文件


## sys

* sys.argv 命令行参数List，第一个元素是程序本身路径
* sys.modules.keys() 返回所有已经导入的模块列表
* sys.exc_info() 获取当前正在处理的异常类,exc_type、exc_value、exc_traceback当前处理的异常详细信息
* sys.exit(n) 退出程序，正常退出时exit(0)
* sys.hexversion 获取Python解释程序的版本值，16进制格式如：0x020403F0
* sys.version 获取Python解释程序的版本信息
* sys.maxint 最大的Int值
* sys.maxunicode 最大的Unicode值
* sys.modules 返回系统导入的模块字段，key是模块名，value是模块
* sys.path 返回模块的搜索路径，初始化时使用PYTHONPATH环境变量的值
* sys.platform 返回操作系统平台名称
* sys.stdout 标准输出
* sys.stdin 标准输入
* sys.stderr 错误输出
* sys.exc_clear() 用来清除当前线程所出现的当前的或最近的错误信息
* sys.exec_prefix 返回平台独立的python文件安装的位置
* sys.byteorder 本地字节规则的指示器，big-endian平台的值是'big',little-endian平台的值是'little'
* sys.copyright 记录python版权相关的东西
* sys.api_version 解释器的C的API版本
* sys.stdin,sys.stdout,sys.stderr

stdin , stdout , 以及stderr 变量包含与标准I/O 流对应的流对象. 如果需要更好地控制输出,而print 不能满足要求, 它们就是所需要的. 你也可以替换它们, 这时候你就可以重定向输出和输入到其它设备( device ), 或者以非标准的方式处理它们

In [102]:
import sys

sys.stdout.write('HelloWorld!')

print('\nPlease enter yourname:')
name = sys.stdin.readline()[:-1]
print('Hi, %s!' % name)

HelloWorld!
Please enter yourname:
Hi, !


stdin, stdout, stderr在Python中无非都是文件属性的对象，他们在Python启动时自动与Shell 环境中的标准输入，输出，出错关联。  

   而Python程序的在Shell中的I/O重定向与本文开始时举的DOS命令的重定向完全相同，其实这种重定向是由Shell来提供的，与Python 本身并无关系。那么我们可以在Python程序内部将stdin,stdout,stderr读写操作重定向到一个内部对象.

Python提供了一个StringIO模块来完成这个设想，Python3中已将StringIO归入io

In [103]:
import sys, io
buff = io.StringIO() 

temp = sys.stdout       # 保存标准I/O流
sys.stdout = buff       # 将标准I/O流重定向到buff对象  StringIO流
print(42, 'hello', 0.001)

sys.stdout = temp       # 恢复标准I/O流
print("获取：", buff.getvalue())

获取： 42 hello 0.001



# 通用补漏

## 闭包函数 - 保护私有变量、编写装饰器函数、

让add（）函数持有一个私有变量--计数器，这样比让变量在全局中暴露来的更加安全。

In [104]:
def add():
    counter = 0
    def plus():
        nonlocal counter 
        counter += 1
        return counter
    return plus   # 闭包函数返回的是一个 内嵌函数

a = add()  # 将变量counter初始化为0，并将内嵌函数提取出来

# 执行内嵌函数时，变量counter不会被add（）函数初始化成 0
print(a())
print(a())
print(a())

1
2
3


可以利用闭包技术，编写装饰器函数，例如计时器

In [105]:
import time

def calcTime(func):
    def calc():
        start = time.time()
        func()
        end = time.time()
        return end-start
    return calc

@calcTime
def f():
    time.sleep(5)

runtime = f()  # f()函数本身并无返回值，是装饰器为他计算、并返回了运行时长
runtime 

5.007656574249268

In [106]:
## 

# 数据结构和算法 

[数据结构与算法之美](https://datastructure.xiaoxiaoming.xyz/#/README?id=%e3%80%8a%e6%95%b0%e6%8d%ae%e7%bb%93%e6%9e%84%e4%b8%8e%e7%ae%97%e6%b3%95%e4%b9%8b%e7%be%8e%e3%80%8b)

# 通用主题实践
- 《python cookbook》

## 数据结构的巧用

### 解压序列赋值给多个变量

#### 问题 
现在有一个包含N 个元素的元组或者是序列，怎样将它里面的值解压后同时赋值
给N 个变量？

#### 解决方案  
任何的序列(或者是可迭代对象) 可以通过一个简单的赋值语句解压并赋值给多个
变量。唯一的前提就是变量的数量必须跟序列元素的数量是一样的。

In [107]:
p = (4, 5)
data = [ 'ACME', 50, 91.1, (2012, 12, 21) ]
x, y  = p
name, shares, price, date = data
a, b, c, d, e = 'hello'

In [108]:
y

5

In [109]:
date

(2012, 12, 21)

In [110]:
d

'l'

#### 讨论  
实际上，这种解压赋值可以用在任何可迭代对象上面，而不仅仅是列表或者元组。
包括字符串，文件对象，迭代器和生成器。

In [111]:
s = "Hello"
a, b, c, d, e = s

In [112]:
a

'H'

有时候，你可能只想解压一部分，丢弃其他的值。对于这种情况Python 并没有提
供特殊的语法。但是你可以使用任意变量名去占位，到时候丢掉这些变量就行了

In [113]:
data = [ 'ACME', 50, 91.1, (2012, 12, 21) ]
_, shares, price, _ = data  # python里想丢掉的一般用 _ 来占位

In [114]:
shares

50

In [115]:
price

91.1

### 解压可迭代对象赋值给多个变量

#### 问题

如果一个可迭代对象的元素个数超过变量个数时，会抛出一个ValueError 。那么
怎样才能从这个可迭代对象中解压出N 个元素出来？

#### 解决方案

Python 的星号表达式可以用来解决这个问题。比如，你在学习一门课程，在学期
末的时候，你想统计下家庭作业的平均成绩，但是排除掉第一个和最后一个分数。如
果只有四个分数，你可能就直接去简单的手动赋值，但如果有24 个呢？这时候星号表
达式就派上用场了：

In [116]:
def drop_first_last(grades):
    first, *middle, last = grades
    return avg(middle)

另外一种情况，假设你现在有一些用户的记录列表，每条记录包含一个名字、邮
件，接着就是不确定数量的电话号码。你可以像下面这样分解这些记录

In [117]:
record = ('Dave', 'dave@example.com', '773-555-1212', '847-555-1212')
name, email, *phone_numbers = record

In [118]:
name

'Dave'

In [119]:
phone_numbers

['773-555-1212', '847-555-1212']

值得注意的是上面解压出的phone numbers 变量永远都是列表类型，不管解压的
电话号码数量是多少(包括0 个)。所以，任何使用到phone numbers 变量的代码就不
需要做多余的类型检查去确认它是否是列表类型了。

星号表达式也能用在列表的开始部分。比如，你有一个公司前8 个月销售数据的序
列，但是你想看下最近一个月数据和前面7 个月的平均值的对比。你可以这样做

```Python
*trailing_qtrs, current_qtr = sales_record
trailing_avg = sum(trailing_qtrs) / len(trailing_qtrs)
return avg_comparison(trailing_avg, current_qtr)
```

In [120]:
*trailing, current = [10, 8, 7, 1, 9, 5, 10, 3]
trailing

[10, 8, 7, 1, 9, 5, 10]

#### 讨论

扩展的迭代解压语法是专门为解压不确定个数或任意个数元素的可迭代对象而设计
的。通常，这些可迭代对象的元素结构有确定的规则（比如第1 个元素后面都是电话
号码），星号表达式让开发人员可以很容易的利用这些规则来解压出元素来。而不是通
过一些比较复杂的手段去获取这些关联的的元素值。  

值得注意的是，星号表达式在迭代元素为可变长元组的序列时是很有用的。比如，
下面是一个带有标签的元组序列：

In [121]:
records = [
    ('foo', 1, 2),
    ('bar', 'hello'),
    ('foo', 3, 4),
]

def do_foo(x, y):
    print('foo', x, y)

def do_bar(s):
    print('bar', s)

for tag, *args in records:
    if tag == 'foo':
        do_foo(*args)
    elif tag == 'bar':
        do_bar(*args)

foo 1 2
bar hello
foo 3 4


星号解压语法在字符串操作的时候也会很有用，比如字符串的分割。

In [122]:
line = 'nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false'
uname, *fields, homedir, sh = line.split(':')

In [123]:
uname

'nobody'

In [124]:
homedir

'/var/empty'

In [125]:
sh

'/usr/bin/false'

有时候，你想解压一些元素后丢弃它们，你不能简单就使用* ，但是你可以使用一
个普通的废弃名称，比如或者ign 。

In [126]:
record = ('ACME', 50, 123.45, (12, 18, 2012))
name, *_, (*_, year) = record

In [127]:
name

'ACME'

In [128]:
year

2012

在很多函数式语言中，星号解压语法跟列表处理有许多相似之处。比如，如果你有
一个列表，你可以很容易的将它分割成前后两部分：

In [129]:
items = [1, 10, 7, 4, 5, 9]
head, *tail = items

In [130]:
head

1

In [131]:
tail

[10, 7, 4, 5, 9]

如果你够聪明的话，还能用这种分割语法去巧妙的实现递归算法

In [132]:
def sum(items):
    head, *tail = items
    return head + sum(tail) if tail else head

sum(items)

36

然后，由于语言层面的限制，递归并不是Python 擅长的。因此，最后那个递归演
示仅仅是个好奇的探索罢了，对这个不要太认真了。

### 保留最后N 个元素

#### 问题   
在迭代操作或者其他操作的时候，怎样只保留最后有限几个元素的历史记录？

#### 解决方案

保留有限历史记录正是collections.deque 大显身手的时候。比如，下面的代码
在多行上面做简单的文本匹配，并只返回在前N 行中匹配成功的行

In [133]:
from collections import deque

def search(lines, pattern, history=5):
    previous_lines = deque(maxlen=history)
    for li in lines:
        if pattern in li:
            yield li, previous_lines
        previous_lines.append(li)
        
# Example use on a file
if __name__ == '__main__':
    with open(r'log.txt') as f:
        for line, prevlines in search(f, 'python', 5):
            for pline in prevlines:
                print(pline, end='')
            print(line, end='')
            print('-' * 20)

python  hello
--------------------
python  hello
java hello
C++ hello
c hello
python  hello
--------------------
c hello
python  hello
java hello
C++ hello
c hello
python  hello
--------------------
c hello
python  hello
java hello
C++ hello
c hello
python  hello
--------------------
c hello
python  hello
java hello
C++ hello
c hello
python  hello
--------------------


#### 讨论

我们在写查询元素的代码时，通常会使用包含yield 表达式的生成器函数，也就是
我们上面示例代码中的那样。这样可以将搜索过程代码和使用搜索结果代码解耦。

使用deque(maxlen=N) 构造函数会新建一个固定大小的队列。当新的元素加入并
且这个队列已满的时候，最老的元素会自动被移除掉。

In [134]:
q = deque(maxlen=3)

In [135]:
q.append(1)

In [136]:
q.append(2)

In [137]:
q.append(3)

In [138]:
q.append(4)

In [139]:
q

deque([2, 3, 4])

In [140]:
q.append(5)
q

deque([3, 4, 5])

尽管你也可以手动在一个列表上实现这一的操作(比如增加、删除等等)。但是这里
的队列方案会更加优雅并且运行得更快些。  
更一般的， deque 类可以被用在任何你只需要一个简单队列数据结构的场合。如
果你不设置最大队列大小，那么就会得到一个无限大小队列，你可以在队列的两端执
行添加和弹出元素的操作。 

In [141]:
q = deque()

In [142]:
q.append(1)

In [143]:
q.append(2)

In [144]:
q.append(3)
q

deque([1, 2, 3])

In [145]:
q.appendleft(4)
q

deque([4, 1, 2, 3])

In [146]:
q.pop()

3

In [147]:
q

deque([4, 1, 2])

In [148]:
q.popleft()

4

In [149]:
q

deque([1, 2])

### 查找最大或最小的N 个元素 

#### 问题

怎样从一个集合中获得最大或者最小的N 个元素列表？

#### 解决方案

heapq 模块有两个函数：nlargest() 和nsmallest() 可以完美解决这个问题。

In [150]:
import heapq
nums = [1, 8, 2, 23, 7, -4, 18, 23, 42, 37, 2]

print(heapq.nlargest(3, nums))

[42, 37, 23]


In [151]:
print(heapq.nsmallest(3, nums))

[-4, 1, 2]


两个函数都能接受一个关键字参数，用于更复杂的数据结构中：

In [152]:
portfolio = [
    {'name': 'IBM', 'shares': 100, 'price': 91.1},
    {'name': 'AAPL', 'shares': 50, 'price': 543.22},
    {'name': 'FB', 'shares': 200, 'price': 21.09},
    {'name': 'HPQ', 'shares': 35, 'price': 31.75},
    {'name': 'YHOO', 'shares': 45, 'price': 16.35},
    {'name': 'ACME', 'shares': 75, 'price': 115.65}
]

代码在对每个元素进行对比的时候，会以price 的值进行比较。

In [153]:
cheap = heapq.nsmallest(3, portfolio, key=lambda s: s["price"])
cheap

[{'name': 'YHOO', 'shares': 45, 'price': 16.35},
 {'name': 'FB', 'shares': 200, 'price': 21.09},
 {'name': 'HPQ', 'shares': 35, 'price': 31.75}]

In [154]:
expensive = heapq.nlargest(3, portfolio, key=lambda s: s["price"])
expensive

[{'name': 'AAPL', 'shares': 50, 'price': 543.22},
 {'name': 'ACME', 'shares': 75, 'price': 115.65},
 {'name': 'IBM', 'shares': 100, 'price': 91.1}]

#### 讨论

如果你想在一个集合中查找最小或最大的N 个元素，并且N 小于集合元素数量，
那么这些函数提供了很好的性能。因为在底层实现里面，首先会先将集合数据进行堆
排序后放入一个列表中：

In [155]:
nums = [1, 8, 2, 23, 7, -4, 18, 23, 42, 37, 2]
import heapq
heapq.heapify(nums)
nums

[-4, 2, 1, 23, 7, 2, 18, 23, 42, 37, 8]

堆数据结构最重要的特征是heap[0] 永远是最小的元素。并且剩余的元素可以很
容易的通过调用heapq.heappop() 方法得到，该方法会先将第一个元素弹出来，然后
用下一个最小的元素来取代被弹出元素(这种操作时间复杂度仅仅是O(log N)，N 是
堆大小)。比如，如果想要查找最小的3 个元素，你可以这样做

In [156]:
heapq.heappop(nums)

-4

In [157]:
heapq.heappop(nums)

1

In [158]:
heapq.heappop(nums)

2

当要查找的元素个数相对比较小的时候，函数nlargest() 和nsmallest() 是很
合适的。如果你仅仅想查找唯一的最小或最大(N=1) 的元素的话，那么使用min() 和
max() 函数会更快些。类似的，如果N 的大小和集合大小接近的时候，通常先排序这
个集合然后再使用切片操作会更快点( sorted(items)[:N] 或者是sorted(items)[-
N:] )。需要在正确场合使用函数nlargest() 和nsmallest() 才能发挥它们的优势(如果
N 快接近集合大小了，那么使用排序操作会更好些)。

尽管你没有必要一定使用这里的方法，但是堆数据结构的实现是一个很有趣并且值
得你深入学习的东西。基本上只要是数据结构和算法书籍里面都会有提及到。

## 字符串和文本

## 数字日期和时间

## 迭代器和生成器

## 文件与 IO

## 数据编码和处理

## 函数

## 面向对象

## 元编程

## 并发程序设计

## 脚本编程与系统管理

## 测试、调试、异常

# 数据库编程
《python核心编程》

# 多线程编程

## threading模块

### 创建一个Thread的实例。传给他一个函数

In [159]:
import threading
from time import sleep, ctime, time

loops = [4, 2]  # 停秒

def loop(nloop, nsec):  # 功能函数
    start = time()
    print("start loop:", nloop)
    sleep(nsec)
    end = time()
    print("loop: %s spend time %s" % (nloop, end-start))
    
def main():
    print("starting at: ", ctime())
    start = time()
    threads = [] # 线程池
    nloops = range(len(loops)) # 线程名
    
    for i in nloops: # 创建线程
        t = threading.Thread(target=loop, args=(i, loops[i])) #新建线程
        threads.append(t) # 向线程池添加
    
    for i in nloops:
        threads[i].start() # 启动线程
    
    for i in nloops:    
        threads[i].join() # 等待线程完成
        
    end = time()
    print("All done take time:", end-start)
    print("all done at: ", ctime())

if __name__=="__main__":
    main()

starting at:  Tue May 19 09:53:02 2020
start loop: 0
start loop: 1
loop: 1 spend time 2.000816583633423
loop: 0 spend time 4.009633779525757
All done take time: 4.009633779525757
all done at:  Tue May 19 09:53:06 2020


###  创建Thread实例，传给它一个可调用的类实例   
（有点尴尬且难以阅读）

In [160]:
import threading
from time import sleep, ctime, time

loops = [4, 2]

class ThreadFunc(object):
    def __init__(self, func, args, name=""):
        self.name = name 
        self.func = func 
        self.args = args
    
    def __call__(self):
        self.func(*self.args)
    
def loop(nloop, nsec):
    start = time()
    print("start loop:", nloop)
    sleep(nsec)
    end = time()
    print("loop: %s spend time %s" % (nloop, end-start))
    
def main():
    print("starting at: ", ctime())
    start = time()
    threads = []
    nloops = range(len(loops))
    
    for i in nloops:
        t = threading.Thread(target=ThreadFunc(loop, (i, loops[i]), loop.__name__))
        threads.append(t)
    
    for i in nloops:
        threads[i].start()
    
    for i in nloops:
        threads[i].join()
    end = time()
    print("All done take time:", end-start)
    print("all done at: ", ctime())

if __name__=="__main__":
    main()

starting at:  Tue May 19 09:53:06 2020
start loop: 0
start loop: 1
loop: 1 spend time 2.0116233825683594
loop: 0 spend time 4.027242660522461
All done take time: 4.027242660522461
all done at:  Tue May 19 09:53:10 2020


### 派生 Thread 的子类，并创建子类的实例。

In [161]:
import threading
from time import sleep, ctime, time

loops = [4, 2]

class MyThread(threading.Thread):
    def __init__(self, func, args, name=""):
        threading.Thread.__init__(self) 
        self.name = name 
        self.func = func 
        self.args = args
    
    def run(self):
        self.func(*self.args)
    
def loop(nloop, nsec):
    start = time()
    print("start loop:", nloop)
    sleep(nsec)
    end = time()
    print("loop: %s spend time %s" % (nloop, end-start))
    
def main():
    print("starting at: ", ctime())
    start = time()
    threads = []
    nloops = range(len(loops))
    
    for i in nloops:
        t = MyThread(loop, (i, loops[i]), loop.__name__)
        threads.append(t)
    
    for i in nloops:
        threads[i].start()
    
    for i in nloops:
        threads[i].join()
    
    end = time()

    print("All done take time:", end-start)
    print("all done at: ", ctime())

if __name__=="__main__":
    main()

starting at:  Tue May 19 09:53:10 2020
start loop: 0
start loop: 1
loop: 1 spend time 2.0304219722747803
loop: 0 spend time 4.000840187072754
All done take time: 4.000840187072754
all done at:  Tue May 19 09:53:14 2020


现在，对MyThread 类进行修改，增加一些调试信息的输出，除了简单地调用函数外，还将把结果保存在实例属性self.res 中，并创建一个新的方法getResult()来获取这个值。

In [162]:
import threading
from time import sleep, ctime, time

loops = [4, 2]

class MyThread(threading.Thread):
    def __init__(self, func, args, name=""):
        threading.Thread.__init__(self) 
        self.name = name 
        self.func = func 
        self.args = args
            
    def getResult(self):
        return self.res
    
    def run(self):
        start = time()
        print("starting Func: ", self.name)
        self.res = self.func(*self.args)
        end = time()
        print(self.name, "Func finished and it take time: ", end-start)
    
def loop(nloop, nsec):

    print("start loop:", nloop)
    sleep(nsec)
    print("loop: %s done !" % (nloop))
    
def main():
    print("starting at: ", ctime())
    start = time()
    threads = []
    nloops = range(len(loops))
    
    for i in nloops:
        t = MyThread(loop, (i, loops[i]), loop.__name__)
        threads.append(t)
    
    for i in nloops:
        threads[i].start()
    
    for i in nloops:
        threads[i].join()
    
    end = time()

    print("All done take time:", end-start)
    print("all done at: ", ctime())

if __name__=="__main__":
    main()

starting at:  Tue May 19 09:53:14 2020
starting Func:  loop
start loop: 0
starting Func:  loop
start loop: 1
loop: 1 done !
loop Func finished and it take time:  2.0146148204803467
loop: 0 done !
loop Func finished and it take time:  4.020639181137085
All done take time: 4.020639181137085
all done at:  Tue May 19 09:53:18 2020


In [163]:
type(loop)

function

## 单线程和多线程执行对比

建立递归求斐波那契、阶乘与累加函数。按
照单线程的方式运行这三个函数。之后使用多线程的方式执行同样的任务，用来说明多线程
环境的优点。

In [164]:
import threading
from time import sleep, ctime, time

loops = [4, 2]

class MyThread(threading.Thread):
    def __init__(self, func, args, name=""):
        threading.Thread.__init__(self) 
        self.name = name 
        self.func = func 
        self.args = args
            
    def getResult(self):
        return self.res
    
    def run(self):
        start = time()
        print("starting: ", self.name)
        self.res = self.func(*self.args)
        end = time()
        print(self.name, "finished and it take time: ", end-start)

def fib(x:int)->int:
    sleep(0.005)
    if x<2 : return 1
    return (fib(x-2)+(fib(x-1)))

def fac(x:int)->int:
    sleep(0.1)
    if x<2: return 1
    return (x* fac(x-1))

def sum(x:int)->int:
    sleep(0.1)
    if x<2 :return 1
    return (x + sum(x-1))

FUNCS = [fib, fac, sum]
N = 12

def main():
    nfuncs = range(len(FUNCS)) #下标池

    print("> START SINGLE THREAD")
    S_start = time()
    for i in nfuncs:
        start = time()
        print(FUNCS[i](N))
        end = time()
        print(FUNCS[i].__name__, "take time :", end-start)
    S_end = time()
    print("> SINGLE THREAD DONE!!\n")
    
    
    print("> START　MULTIPLE THREADS")
    M_start = time()
    threads = [] # 线程池
    for i in nfuncs:
        t = MyThread(FUNCS[i], (N, ), FUNCS[i].__name__)
        threads.append(t)
    
    for i in nfuncs: 
        threads[i].start()
    
    for i in nfuncs:
        threads[i].join()
        print(threads[i].getResult())   
    M_end = time()
    print("> MULTIPLE THREADS DONE!!!\n")
    

    print("*** SINGLE THREAD take time: ", S_end-S_start)
    print("*** MULTIPLE THREADS take time: ", M_end-M_start)        

if __name__=="__main__":
    main()

> START SINGLE THREAD
233
fib take time : 5.675452709197998
479001600
fac take time : 1.2792155742645264
78
sum take time : 1.2634167671203613
> SINGLE THREAD DONE!!

> START　MULTIPLE THREADS
starting:  fib
starting:  fac
starting:  sum
fac finished and it take time:  1.3006129264831543
sum finished and it take time:  1.3162128925323486
fib finished and it take time:  5.657253742218018
233
479001600
78
> MULTIPLE THREADS DONE!!!

*** SINGLE THREAD take time:  8.218085050582886
*** MULTIPLE THREADS take time:  5.657253742218018


# 网络编程入门
摘自 [github](https://github.com/jackfrued/Python-100-Days/blob/master/Day01-15/14.%E7%BD%91%E7%BB%9C%E7%BC%96%E7%A8%8B%E5%85%A5%E9%97%A8%E5%92%8C%E7%BD%91%E7%BB%9C%E5%BA%94%E7%94%A8%E5%BC%80%E5%8F%91.md)

## 计算机网络基础

计算机网络是独立自主的计算机互联而成的系统的总称，组建计算机网络最主要的目的是实现多台计算机之间的通信和资源共享。今天计算机网络中的设备和计算机网络的用户已经多得不可计数，而计算机网络也可以称得上是一个“复杂巨系统”，对于这样的系统，我们不可能用一两篇文章把它讲清楚，有兴趣的读者可以自行阅读Andrew S.Tanenbaum老师的经典之作《计算机网络》或Kurose和Ross老师合著的《计算机网络:自顶向下方法》来了解计算机网络的相关知识。

### 计算机网络发展史

1. 1960s - 美国国防部ARPANET项目问世，奠定了分组交换网络的基础。

   ![](./res/arpanet.png)

2. 1980s - 国际标准化组织（ISO）发布OSI/RM，奠定了网络技术标准化的基础。

   ![](./res/osimodel.png)

3. 1990s - 英国人[蒂姆·伯纳斯-李](https://zh.wikipedia.org/wiki/%E6%8F%90%E5%A7%86%C2%B7%E6%9F%8F%E5%85%A7%E8%8C%B2-%E6%9D%8E)发明了图形化的浏览器，浏览器的简单易用性使得计算机网络迅速被普及。

   在没有浏览器的年代，上网是这样的。

   ![](./res/before-browser.jpg)

   有了浏览器以后，上网是这样的。

   ![](./res/after-browser.jpg)

### TCP/IP模型

实现网络通信的基础是网络通信协议，这些协议通常是由[互联网工程任务组](https://zh.wikipedia.org/wiki/%E4%BA%92%E8%81%94%E7%BD%91%E5%B7%A5%E7%A8%8B%E4%BB%BB%E5%8A%A1%E7%BB%84) （IETF）制定的。所谓“协议”就是通信计算机双方必须共同遵从的一组约定，例如怎样建立连接、怎样互相识别等，网络协议的三要素是：语法、语义和时序。构成我们今天使用的Internet的基础的是TCP/IP协议族，所谓协议族就是一系列的协议及其构成的通信模型，我们通常也把这套东西称为TCP/IP模型。与国际标准化组织发布的OSI/RM这个七层模型不同，TCP/IP是一个四层模型，也就是说，该模型将我们使用的网络从逻辑上分解为四个层次，自底向上依次是：网络接口层、网络层、传输层和应用层，如下图所示。

![](./res/TCP-IP-model.png)

IP通常被翻译为网际协议，它服务于网络层，主要实现了寻址和路由的功能。接入网络的每一台主机都需要有自己的IP地址，IP地址就是主机在计算机网络上的身份标识。当然由于IPv4地址的匮乏，我们平常在家里、办公室以及其他可以接入网络的公共区域上网时获得的IP地址并不是全球唯一的IP地址，而是一个[局域网（LAN）](https://zh.wikipedia.org/zh-hans/%E5%B1%80%E5%9F%9F%E7%BD%91)中的内部IP地址，通过[网络地址转换（NAT）服务](https://zh.wikipedia.org/wiki/%E7%BD%91%E7%BB%9C%E5%9C%B0%E5%9D%80%E8%BD%AC%E6%8D%A2)我们也可以实现对网络的访问。计算机网络上有大量的被我们称为“[路由器](https://zh.wikipedia.org/wiki/%E8%B7%AF%E7%94%B1%E5%99%A8)”的网络中继设备，它们会存储转发我们发送到网络上的数据分组，让从源头发出的数据最终能够找到传送到目的地通路，这项功能就是所谓的路由。

TCP全称传输控制协议，它是基于IP提供的寻址和路由服务而建立起来的负责实现端到端可靠传输的协议，之所以将TCP称为可靠的传输协议是因为TCP向调用者承诺了三件事情：

1. 数据不传丢不传错（利用握手、校验和重传机制可以实现）。
2. 流量控制（通过滑动窗口匹配数据发送者和接收者之间的传输速度）。
3. 拥塞控制（通过RTT时间以及对滑动窗口的控制缓解网络拥堵）。

### 网络应用模式

1. C/S模式和B/S模式。这里的C指的是Client（客户端），通常是一个需要安装到某个宿主操作系统上的应用程序；而B指的是Browser（浏览器），它几乎是所有图形化操作系统都默认安装了的一个应用软件；通过C或B都可以实现对S（服务器）的访问。关于二者的比较和讨论在网络上有一大堆的文章，在此我们就不再浪费笔墨了。
2. 去中心化的网络应用模式。不管是B/S还是C/S都需要服务器的存在，服务器就是整个应用模式的中心，而去中心化的网络应用通常没有固定的服务器或者固定的客户端，所有应用的使用者既可以作为资源的提供者也可以作为资源的访问者。

## 基于HTTP协议的网络资源访问

### HTTP（超文本传输协议）

HTTP是超文本传输协议（Hyper-Text Transfer Proctol）的简称，维基百科上对HTTP的解释是：超文本传输协议是一种用于分布式、协作式和超媒体信息系统的应用层协议，它是[万维网](https://zh.wikipedia.org/wiki/%E5%85%A8%E7%90%83%E8%B3%87%E8%A8%8A%E7%B6%B2)数据通信的基础，设计HTTP最初的目的是为了提供一种发布和接收[HTML](https://zh.wikipedia.org/wiki/HTML)页面的方法，通过HTTP或者[HTTPS](https://zh.wikipedia.org/wiki/%E8%B6%85%E6%96%87%E6%9C%AC%E4%BC%A0%E8%BE%93%E5%AE%89%E5%85%A8%E5%8D%8F%E8%AE%AE)（超文本传输安全协议）请求的资源由URI（[统一资源标识符](https://zh.wikipedia.org/wiki/%E7%B5%B1%E4%B8%80%E8%B3%87%E6%BA%90%E6%A8%99%E8%AD%98%E7%AC%A6)）来标识。关于HTTP的更多内容，我们推荐阅读阮一峰老师的[《HTTP 协议入门》](http://www.ruanyifeng.com/blog/2016/08/http.html)，简单的说，通过HTTP我们可以获取网络上的（基于字符的）资源，开发中经常会用到的网络API（有的地方也称之为网络数据接口）就是基于HTTP来实现数据传输的。

### JSON格式

**JSON**（**J**ava**S**cript **O**bject **N**otation）是一种轻量级的数据交换语言，该语言以易于让人阅读的文字（纯文本）为基础，用来传输由属性值或者序列性的值组成的数据对象。尽管JSON是最初只是Javascript中一种创建对象的字面量语法，但它在当下更是一种独立于语言的数据格式，很多编程语言都支持JSON格式数据的生成和解析，Python内置的json模块也提供了这方面的功能。由于JSON是纯文本，它和[XML](https://zh.wikipedia.org/wiki/XML)一样都适用于异构系统之间的数据交换，而相较于XML，JSON显得更加的轻便和优雅。下面是表达同样信息的XML和JSON，而JSON的优势是相当直观的。

XML的例子：

```XML
<?xml version="1.0" encoding="UTF-8"?>
<message>
	<from>Alice</from>
	<to>Bob</to>
	<content>Will you marry me?</content>
</message>
```

JSON的例子：

```JSON
{
    "from": "Alice",
    "to": "Bob",
    "content": "Will you marry me?"
}
```

### requests库

requests是一个基于HTTP协议来使用网络的第三库，其[官方网站](http://cn.python-requests.org/zh_CN/latest/)有这样的一句介绍它的话：“Requests是唯一的一个**非转基因**的Python HTTP库，人类可以安全享用。”简单的说，使用requests库可以非常方便的使用HTTP，避免安全缺陷、冗余代码以及“重复发明轮子”（行业黑话，通常用在软件工程领域表示重新创造一个已有的或是早已被优化過的基本方法）。前面的文章中我们已经使用过这个库，下面我们还是通过requests来实现一个访问网络数据接口并从中获取美女图片下载链接然后下载美女图片到本地的例子程序，程序中使用了[天行数据](https://www.tianapi.com/)提供的网络API。

我们可以先通过pip安装requests及其依赖库。

```Shell
pip install requests
```

如果使用PyCharm作为开发工具，可以直接在代码中书写`import requests`，然后通过代码修复功能来自动下载安装requests。

```Python
from time import time
from threading import Thread

import requests


# 继承Thread类创建自定义的线程类
class DownloadHanlder(Thread):

    def __init__(self, url):
        super().__init__()
        self.url = url

    def run(self):
        filename = self.url[self.url.rfind('/') + 1:]
        resp = requests.get(self.url)
        with open('/Users/Hao/' + filename, 'wb') as f:
            f.write(resp.content)


def main():
    # 通过requests模块的get函数获取网络资源
    # 下面的代码中使用了天行数据接口提供的网络API
    # 要使用该数据接口需要在天行数据的网站上注册
    # 然后用自己的Key替换掉下面代码的中APIKey即可
    resp = requests.get(
        'http://api.tianapi.com/meinv/?key=APIKey&num=10')
    # 将服务器返回的JSON格式的数据解析为字典
    data_model = resp.json()
    for mm_dict in data_model['newslist']:
        url = mm_dict['picUrl']
        # 通过多线程的方式实现图片下载
        DownloadHanlder(url).start()


if __name__ == '__main__':
    main()
```

## 基于传输层协议的套接字编程

套接字这个词对很多不了解网络编程的人来说显得非常晦涩和陌生，其实说得通俗点，套接字就是一套用[C语言](https://zh.wikipedia.org/wiki/C%E8%AF%AD%E8%A8%80)写成的应用程序开发库，主要用于实现进程间通信和网络编程，在网络应用开发中被广泛使用。在Python中也可以基于套接字来使用传输层提供的传输服务，并基于此开发自己的网络应用。实际开发中使用的套接字可以分为三类：流套接字（TCP套接字）、数据报套接字和原始套接字。

### 套接字  - 摘自《python核心编程》
套接字是计算机网络数据结构，它体现了上节中所描述的“通信端点”的概念。在任何
类型的通信开始之前，网络应用程序必须创建套接字。可以将它们比作电话插孔，没有它将
无法进行通信。  

套接字的起源可以追溯到20 世纪70 年代，它是加利福尼亚大学的伯克利版本UNIX（称
为BSD UNIX）的一部分。因此，有时你可能会听过将套接字称为伯克利套接字或BSD 套接
字。套接字最初是为同一主机上的应用程序所创建，使得主机上运行的一个程序（又名一个
进程）与另一个运行的程序进行通信。这就是所谓的进程间通信（Inter Process Communication，
IPC）。有两种类型的套接字：基于文件的和面向网络的。  

UNIX 套接字是我们所讲的套接字的第一个家族，并且拥有一个“家族名字”AF_UNIX
（又名AF_LOCAL，在POSIX1.g 标准中指定），它代表地址家族（address family）：UNIX。
包括Python 在内的大多数受欢迎的平台都使用术语地址家族及其缩写AF；其他比较旧的系
统可能会将地址家族表示成域（domain）或协议家族（protocol family），并使用其缩写PF 而
非AF。类似地，AF_LOCAL（在2000～2001 年标准化）将代替AF_UNIX。然而，考虑到
后向兼容性，很多系统都同时使用二者，只是对同一个常数使用不同的别名。Python 本身仍
然在使用AF_UNIX。  

因为两个进程运行在同一台计算机上，所以这些套接字都是基于文件的，这意味着文件
系统支持它们的底层基础结构。这是能够说得通的，因为文件系统是一个运行在同一主机上
的多个进程之间的共享常量。  

第二种类型的套接字是基于网络的，它也有自己的家族名字AF_INET，或者地址家族：
因特网。另一个地址家族AF_INET6 用于第6 版因特网协议（IPv6）寻址。此外，还有其他
的地址家族，这些要么是专业的、过时的、很少使用的，要么是仍未实现的。在所有的地址
家族之中，目前AF_INET 是使用得最广泛的。  

Python 2.5 中引入了对特殊类型的Linux 套接字的支持。套接字的AF_NETLINK 家族（无
连接[见2.3.3 节]）允许使用标准的BSD 套接字接口进行用户级别和内核级别代码之间的IPC。
之前那种解决方案比较麻烦，而这个解决方案可以看作一种比前一种更加优雅且风险更低的
解决方案，例如，添加新系统调用、/proc 支持，或者对一个操作系统的“IOCTL”。  

针对 Linux 的另一种特性（Python 2.6 中新增）就是支持透明的进程间通信（TIPC）协
议。TIPC 允许计算机集群之中的机器相互通信，而无须使用基于IP 的寻址方式。Python 对
TIPC 的支持以AF_TIPC 家族的方式呈现。  

总的来说，Python 只支持AF_UNIX、AF_NETLINK、AF_TIPC 和AF_INET 家族。

### TCP套接字

所谓TCP套接字就是使用TCP协议提供的传输服务来实现网络通信的编程接口。在Python中可以通过创建socket对象并指定type属性为SOCK_STREAM来使用TCP套接字。由于一台主机可能拥有多个IP地址，而且很有可能会配置多个不同的服务，所以作为服务器端的程序，需要在创建套接字对象后将其绑定到指定的IP地址和端口上。这里的端口并不是物理设备而是对IP地址的扩展，用于区分不同的服务，例如我们通常将HTTP服务跟80端口绑定，而MySQL数据库服务默认绑定在3306端口，这样当服务器收到用户请求时就可以根据端口号来确定到底用户请求的是HTTP服务器还是数据库服务器提供的服务。端口的取值范围是0~65535，而1024以下的端口我们通常称之为“著名端口”（留给像FTP、HTTP、SMTP等“著名服务”使用的端口，有的地方也称之为“周知端口”），自定义的服务通常不使用这些端口，除非自定义的是HTTP或FTP这样的著名服务。

下面的代码实现了一个提供时间日期的服务器。

```Python
from socket import socket, SOCK_STREAM, AF_INET
from datetime import datetime


def main():
    # 1.创建套接字对象并指定使用哪种传输服务
    # family=AF_INET - IPv4地址
    # family=AF_INET6 - IPv6地址
    # type=SOCK_STREAM - TCP套接字
    # type=SOCK_DGRAM - UDP套接字
    # type=SOCK_RAW - 原始套接字
    server = socket(family=AF_INET, type=SOCK_STREAM)
    # 2.绑定IP地址和端口(端口用于区分不同的服务)
    # 同一时间在同一个端口上只能绑定一个服务否则报错
    server.bind(('127.0.0.1', 6789))
    # 3.开启监听 - 监听客户端连接到服务器
    # 参数512可以理解为连接队列的大小
    server.listen(512)
    print('服务器启动开始监听...')
    while True:
        # 4.通过循环接收客户端的连接并作出相应的处理(提供服务)
        # accept方法是一个阻塞方法如果没有客户端连接到服务器代码不会向下执行
        # accept方法返回一个元组其中的第一个元素是客户端对象
        # 第二个元素是连接到服务器的客户端的地址(由IP和端口两部分构成)
        client, addr = server.accept()
        print(str(addr) + '连接到了服务器.')
        # 5.发送数据
        client.send(str(datetime.now()).encode('utf-8'))
        # 6.断开连接
        client.close()


if __name__ == '__main__':
    main()
```

运行服务器程序后我们可以通过Windows系统的telnet来访问该服务器，结果如下图所示。

```Shell
telnet 127.0.0.1  6789
```

![](./res/telnet.png)

当然我们也可以通过Python的程序来实现TCP客户端的功能，相较于实现服务器程序，实现客户端程序就简单多了，代码如下所示。

```Python
from socket import socket


def main():
    # 1.创建套接字对象默认使用IPv4和TCP协议
    client = socket()
    # 2.连接到服务器(需要指定IP地址和端口)
    client.connect(('127.0.0.1', 6789))
    # 3.从服务器接收数据
    print(client.recv(1024).decode('utf-8'))
    client.close()


if __name__ == '__main__':
    main()
```

需要注意的是，上面的服务器并没有使用多线程或者异步I/O的处理方式，这也就意味着当服务器与一个客户端处于通信状态时，其他的客户端只能排队等待。很显然，这样的服务器并不能满足我们的需求，我们需要的服务器是能够同时接纳和处理多个用户请求的。下面我们来设计一个使用多线程技术处理多个用户请求的服务器，该服务器会向连接到服务器的客户端发送一张图片。

服务器端代码：

```Python
from socket import socket, SOCK_STREAM, AF_INET
from base64 import b64encode
from json import dumps
from threading import Thread


def main():
    
    # 自定义线程类
    class FileTransferHandler(Thread):

        def __init__(self, cclient):
            super().__init__()
            self.cclient = cclient

        def run(self):
            my_dict = {}
            my_dict['filename'] = 'guido.jpg'
            # JSON是纯文本不能携带二进制数据
            # 所以图片的二进制数据要处理成base64编码
            my_dict['filedata'] = data
            # 通过dumps函数将字典处理成JSON字符串
            json_str = dumps(my_dict)
            # 发送JSON字符串
            self.cclient.send(json_str.encode('utf-8'))
            self.cclient.close()

    # 1.创建套接字对象并指定使用哪种传输服务
    server = socket()
    # 2.绑定IP地址和端口(区分不同的服务)
    server.bind(('127.0.0.1', 5566))
    # 3.开启监听 - 监听客户端连接到服务器
    server.listen(512)
    print('服务器启动开始监听...')
    with open('guido.jpg', 'rb') as f:
        # 将二进制数据处理成base64再解码成字符串
        data = b64encode(f.read()).decode('utf-8')
    while True:
        client, addr = server.accept()
        # 启动一个线程来处理客户端的请求
        FileTransferHandler(client).start()


if __name__ == '__main__':
    main()
```

客户端代码：

```Python
from socket import socket
from json import loads
from base64 import b64decode


def main():
    client = socket()
    client.connect(('127.0.0.1', 5566))
    # 定义一个保存二进制数据的对象
    in_data = bytes()
    # 由于不知道服务器发送的数据有多大每次接收1024字节
    data = client.recv(1024)
    while data:
        # 将收到的数据拼接起来
        in_data += data
        data = client.recv(1024)
    # 将收到的二进制数据解码成JSON字符串并转换成字典
    # loads函数的作用就是将JSON字符串转成字典对象
    my_dict = loads(in_data.decode('utf-8'))
    filename = my_dict['filename']
    filedata = my_dict['filedata'].encode('utf-8')
    with open('/Users/Hao/' + filename, 'wb') as f:
        # 将base64格式的数据解码成二进制数据并写入文件
        f.write(b64decode(filedata))
    print('图片已保存.')


if __name__ == '__main__':
    main()
```

在这个案例中，我们使用了JSON作为数据传输的格式（通过JSON格式对传输的数据进行了序列化和反序列化的操作），但是JSON并不能携带二进制数据，因此对图片的二进制数据进行了Base64编码的处理。Base64是一种用64个字符表示所有二进制数据的编码方式，通过将二进制数据每6位一组的方式重新组织，刚好可以使用0~9的数字、大小写字母以及“+”和“/”总共64个字符表示从`000000`到`111111`的64种状态。[维基百科](https://zh.wikipedia.org/wiki/Base64)上有关于Base64编码的详细讲解，不熟悉Base64的读者可以自行阅读。

> **说明：** 上面的代码主要为了讲解网络编程的相关内容因此并没有对异常状况进行处理，请读者自行添加异常处理代码来增强程序的健壮性。

### UDP套接字

传输层除了有可靠的传输协议TCP之外，还有一种非常轻便的传输协议叫做用户数据报协议，简称UDP。TCP和UDP都是提供端到端传输服务的协议，二者的差别就如同打电话和发短信的区别，后者不对传输的可靠性和可达性做出任何承诺从而避免了TCP中握手和重传的开销，所以在强调性能和而不是数据完整性的场景中（例如传输网络音视频数据），UDP可能是更好的选择。可能大家会注意到一个现象，就是在观看网络视频时，有时会出现卡顿，有时会出现花屏，这无非就是部分数据传丢或传错造成的。在Python中也可以使用UDP套接字来创建网络应用，对此我们不进行赘述，有兴趣的读者可以自行研究。

## 网络应用开发

### 发送电子邮件

在即时通信软件如此发达的今天，电子邮件仍然是互联网上使用最为广泛的应用之一，公司向应聘者发出录用通知、网站向用户发送一个激活账号的链接、银行向客户推广它们的理财产品等几乎都是通过电子邮件来完成的，而这些任务应该都是由程序自动完成的。

就像我们可以用HTTP（超文本传输协议）来访问一个网站一样，发送邮件要使用SMTP（简单邮件传输协议），SMTP也是一个建立在TCP（传输控制协议）提供的可靠数据传输服务的基础上的应用级协议，它规定了邮件的发送者如何跟发送邮件的服务器进行通信的细节，而Python中的smtplib模块将这些操作简化成了几个简单的函数。

下面的代码演示了如何在Python发送邮件。

```Python
from smtplib import SMTP
from email.header import Header
from email.mime.text import MIMEText


def main():
    # 请自行修改下面的邮件发送者和接收者
    sender = 'abcdefg@126.com'
    receivers = ['uvwxyz@qq.com', 'uvwxyz@126.com']
    message = MIMEText('用Python发送邮件的示例代码.', 'plain', 'utf-8')
    message['From'] = Header('王大锤', 'utf-8')
    message['To'] = Header('骆昊', 'utf-8')
    message['Subject'] = Header('示例代码实验邮件', 'utf-8')
    smtper = SMTP('smtp.126.com')
    # 请自行修改下面的登录口令
    smtper.login(sender, 'secretpass')
    smtper.sendmail(sender, receivers, message.as_string())
    print('邮件发送完成!')


if __name__ == '__main__':
    main()
```

如果要发送带有附件的邮件，那么可以按照下面的方式进行操作。

```Python
from smtplib import SMTP
from email.header import Header
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

import urllib


def main():
    # 创建一个带附件的邮件消息对象
    message = MIMEMultipart()
    
    # 创建文本内容
    text_content = MIMEText('附件中有本月数据请查收', 'plain', 'utf-8')
    message['Subject'] = Header('本月数据', 'utf-8')
    # 将文本内容添加到邮件消息对象中
    message.attach(text_content)

    # 读取文件并将文件作为附件添加到邮件消息对象中
    with open('/Users/Hao/Desktop/hello.txt', 'rb') as f:
        txt = MIMEText(f.read(), 'base64', 'utf-8')
        txt['Content-Type'] = 'text/plain'
        txt['Content-Disposition'] = 'attachment; filename=hello.txt'
        message.attach(txt)
    # 读取文件并将文件作为附件添加到邮件消息对象中
    with open('/Users/Hao/Desktop/汇总数据.xlsx', 'rb') as f:
        xls = MIMEText(f.read(), 'base64', 'utf-8')
        xls['Content-Type'] = 'application/vnd.ms-excel'
        xls['Content-Disposition'] = 'attachment; filename=month-data.xlsx'
        message.attach(xls)
    
    # 创建SMTP对象
    smtper = SMTP('smtp.126.com')
    # 开启安全连接
    # smtper.starttls()
    sender = 'abcdefg@126.com'
    receivers = ['uvwxyz@qq.com']
    # 登录到SMTP服务器
    # 请注意此处不是使用密码而是邮件客户端授权码进行登录
    # 对此有疑问的读者可以联系自己使用的邮件服务器客服
    smtper.login(sender, 'secretpass')
    # 发送邮件
    smtper.sendmail(sender, receivers, message.as_string())
    # 与邮件服务器断开连接
    smtper.quit()
    print('发送完成!')


if __name__ == '__main__':
    main()
```

### 发送短信

发送短信也是项目中常见的功能，网站的注册码、验证码、营销信息基本上都是通过短信来发送给用户的。在下面的代码中我们使用了[互亿无线](http://www.ihuyi.com/)短信平台（该平台为注册用户提供了50条免费短信以及常用开发语言发送短信的demo，可以登录该网站并在用户自服务页面中对短信进行配置）提供的API接口实现了发送短信的服务，当然国内的短信平台很多，读者可以根据自己的需要进行选择（通常会考虑费用预算、短信达到率、使用的难易程度等指标），如果需要在商业项目中使用短信服务建议购买短信平台提供的套餐服务。

```Python
import urllib.parse
import http.client
import json


def main():
    host  = "106.ihuyi.com"
    sms_send_uri = "/webservice/sms.php?method=Submit"
    # 下面的参数需要填入自己注册的账号和对应的密码
    params = urllib.parse.urlencode({'account': '你自己的账号', 'password' : '你自己的密码', 'content': '您的验证码是：147258。请不要把验证码泄露给其他人。', 'mobile': '接收者的手机号', 'format':'json' })
    print(params)
    headers = {'Content-type': 'application/x-www-form-urlencoded', 'Accept': 'text/plain'}
    conn = http.client.HTTPConnection(host, port=80, timeout=30)
    conn.request('POST', sms_send_uri, params, headers)
    response = conn.getresponse()
    response_str = response.read()
    jsonstr = response_str.decode('utf-8')
    print(json.loads(jsonstr))
    conn.close()


if __name__ == '__main__':
    main()
```